# Projet streamlit - Partie Modèles

## Projet Supply Chain - DSAvril23

## 1. Import des packages et données

In [3]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

In [4]:
from sklearn import model_selection
from sklearn import ensemble
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 

from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import pickle
from tensorflow.keras.models import save_model
from tensorflow.keras.models import load_model

In [3]:
pip install imblearn


  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.11.0-py3-none-any.whl.metadata (8.3 kB)
Using cached imbalanced_learn-0.11.0-py3-none-any.whl (235 kB)


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
# cellule à relancer lors de la prochaine ré-ouverture du notebook
df=pd.read_csv('Compilation webscrapping cosmetique.csv')

In [21]:
# dans cette partie modélisation, seules les variables commentaires et notes nous intéressent
# transformation du type de la note
df['notes']=df['notes'].astype('int')

In [8]:
# On ne garde de notre dataframe que les commentaires (variable explicative) et les notes (variable à prédire)
X, y = df.commentaire, df.notes

# Séparation des données d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Définition de l'objet CountVectorizer() et du GradientBoostingClassifier()
vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [38]:
# Enregistrement de l'objet vectorizer
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [9]:
smo = SMOTE()
X_sm, y_sm = smo.fit_resample(X_train, y_train)
print('Classes échantillon SMOTE :', dict(pd.Series(y_sm).value_counts()))

Classes échantillon SMOTE : {1: 2667, 5: 2667, 4: 2667, 2: 2667, 3: 2667}


## 2. Random forest avec smote

In [11]:
## RANDOM FOREST SUR JEU COMPLET APRES SMOTE

rfc = RandomForestClassifier()

params_rfc = {"max_features":["sqrt","log2",None], "min_samples_split" : [x for x in range(2,31) if x%2 ==0]}
grid_rfc = GridSearchCV(estimator=rfc, param_grid=params_rfc, cv=3)
grid_rfc.fit(X_train, y_train)

# Afficher les meilleurs paramètres du Random Forest
print("Les meilleurs paramètres du Random Forest sont :\n", grid_rfc.best_params_)

y_pred_rfc_train = grid_rfc.predict(X_train)
y_pred_rfc_test = grid_rfc.predict(X_test)

# Afficher le classification report du modèle rfc sur les données d'entrainement
print("Le classification report du modèle rfc sur les données d'entrainement est :\n", classification_report(y_train, y_pred_rfc_train))

# Afficher le classification report du modèle rfc sur les données de test
print("Le classification report du modèle rfc sur les données de test est :\n", classification_report(y_test, y_pred_rfc_test))

# Afficher la matrice de confusion sur l'ensemble de test
print("La matrice de confusion sur l'ensemble de test est :\n", pd.crosstab(y_test, y_pred_rfc_test))

Les meilleurs paramètres du Random Forest sont :
 {'max_features': 'log2', 'min_samples_split': 26}
Le classification report du modèle rfc sur les données d'entrainement est :
               precision    recall  f1-score   support

           1       0.86      0.96      0.91      1287
           2       1.00      0.13      0.24       164
           3       1.00      0.09      0.17       132
           4       1.00      0.17      0.28       315
           5       0.87      1.00      0.93      2659

    accuracy                           0.87      4557
   macro avg       0.95      0.47      0.50      4557
weighted avg       0.89      0.87      0.83      4557

Le classification report du modèle rfc sur les données de test est :
               precision    recall  f1-score   support

           1       0.80      0.81      0.80       299
           2       0.00      0.00      0.00        38
           3       0.00      0.00      0.00        45
           4       0.00      0.00      0.00    

In [14]:
# Enregistrement du modèle rfc
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(grid_rfc, file)

## 3. SVC avec smote

In [17]:
## SVC SUR JEU COMPLET APRES SMOTE

svc = SVC()

params_svc = {"kernel":["linear","sigmoid","rbf"], "C" : [0.1, 1, 10, 30]}
grid_svc=GridSearchCV(estimator = svc, param_grid = params_svc, cv=5)
grid_svc.fit(X_train, y_train)

# Afficher les meilleurs paramètres du SVC
print("Les meilleurs paramètres du SVC sont :\n", grid_svc.best_params_)

y_pred_svc_train = grid_svc.predict(X_train)
y_pred_svc_test = grid_svc.predict(X_test)
    
# Afficher le classification report du modèle SVC sur les données d'entrainement
print("Le classification report du modèle SVC sur les données d'entrainement est :\n", classification_report(y_train, y_pred_svc_train))

# Afficher le classification report du modèle SVC sur les données de test
print("Le classification report du modèle SVC sur les données de test est :\n", classification_report(y_test, y_pred_svc_test))

# Afficher la matrice de confusion sur l'ensemble de test
print("La matrice de confusion sur l'ensemble de test est :\n", pd.crosstab(y_test, y_pred_svc_test)) 

Les meilleurs paramètres du SVC sont :
 {'C': 10, 'kernel': 'rbf'}
Le classification report du modèle SVC sur les données d'entrainement est :
               precision    recall  f1-score   support

           1       1.00      0.97      0.98      1287
           2       1.00      0.94      0.97       164
           3       1.00      0.92      0.96       132
           4       1.00      0.72      0.84       315
           5       0.95      1.00      0.97      2659

    accuracy                           0.97      4557
   macro avg       0.99      0.91      0.95      4557
weighted avg       0.97      0.97      0.97      4557

Le classification report du modèle SVC sur les données de test est :
               precision    recall  f1-score   support

           1       0.77      0.90      0.83       299
           2       0.25      0.05      0.09        38
           3       0.20      0.02      0.04        45
           4       0.32      0.08      0.12        90
           5       0.85   

In [18]:
# Enregistrement du modèle svc
with open('svc_model.pkl', 'wb') as file:
    pickle.dump(grid_svc, file)

## 4. Word embedding

In [22]:
# AttributeError: Can only use .cat accessor with a 'category' dtype
df=pd.read_csv('Compilation webscrapping cosmetique.csv')
df['notes']=df['notes'].astype('int')
df['notes'] = df['notes'].astype('category')

In [25]:
## EMBEDDING SUR JEU COMPLET

X, y = df.commentaire, df.notes

# Séparation des données d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

num_classes = len(df['notes'].cat.categories)
max_words = 1000  
max_len = 100 

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_text = tokenizer.texts_to_sequences(X_train)
X_test_text = tokenizer.texts_to_sequences(X_test)

train_text = pad_sequences(X_train_text, maxlen=max_len)
test_text = pad_sequences(X_test_text, maxlen=max_len)

embedding_dim = 50

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher le summary du modèle d'Embedding
print("Le summary du modèle d'Embedding est :\n", model.summary()) 
    
y_train_cat = to_categorical(y_train.cat.codes)
y_test_cat = to_categorical(y_test.cat.codes)

batch_size = 32
epochs = 5

model.fit(train_text, y_train_cat, epochs=epochs, batch_size=batch_size, validation_split=0.2)

loss, accuracy = model.evaluate(test_text, y_test_cat)
print("L accuracy sur notre jeu de test est : ", accuracy)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 50)           50000     
                                                                 
 spatial_dropout1d_3 (Spati  (None, 100, 50)           0         
 alDropout1D)                                                    
                                                                 
 lstm_3 (LSTM)               (None, 100)               60400     
                                                                 
 dense_3 (Dense)             (None, 5)                 505       
                                                                 
Total params: 110905 (433.22 KB)
Trainable params: 110905 (433.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Le summary du modèle d'Embedding est :
 None
Epoch 1/5
114/114 [================

In [26]:
# Enregistrement du modèle d'embedding
model.save('embedding_model.h5') #ne fonctionne pas avec pickle

In [28]:
# Enregistrement du tokenizer
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)